In [1]:
import os
from data_preparation import *
from model import *


In [2]:
BATCH_SIZE = 32
BUFFER_SIZE = 10000
dataset, char2idx, idx2char, vocab = word_text2tf_data(filename='donald_tweets.txt',
                                                       seq_length=20,
                                                       batch_size=BATCH_SIZE,
                                                       buffer_size=BUFFER_SIZE)

In [3]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNIT = 64
model = build_attention_model(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_units=RNN_UNIT,
    batch_size=BATCH_SIZE)
model.summary()

Model: "ModelWithAttention"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(32, None)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (32, None, 256)      5555712     input_1[0][0]                    
                                                                 input_1[0][0]                    
__________________________________________________________________________________________________
gru (GRU)                       (32, None, 64)       61824       embedding[0][0]                  
                                                                 embedding[1][0]                  
_________________________________________________________________________________

In [4]:
LOSS = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
OPTIMIZER = 'adam'
model.compile(optimizer=OPTIMIZER, loss=LOSS)

In [5]:
checkpoint_dir = './attention_word_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 10
history = model.fit(dataset,
                    epochs=EPOCHS,
                    callbacks=[checkpoint_callback])
tf.train.latest_checkpoint(checkpoint_dir)

Epoch 1/10
192/192 [==============================] - 7s 38ms/step - loss: 8.1095

'./attention_word_checkpoints\\ckpt_10'

In [6]:
model = build_attention_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNIT, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))


In [7]:
def generate_text(model, start_string, num_generate, split_string):
    start_string
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)


    text_generated = []
    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return split_string.join(start_string) + split_string.join(text_generated)


print(generate_text(model, ['@ABCPolitics:'], 1000, ' '))

@ABCPolitics:Dream! yet! 
 Happy How America needs a lot of him. Bad IN NOMINATION! #TrumpTrain #Trump2016 #ICYMI, #Trump! _" 
 .@BrandenRoderick I will MAKE AMERICA GREAT AGAIN! 
 Getting one of American need to President Rhode Rapids, South Carolina! Nice 
 @oreillyfactor, she۪s doing almost someone that needs to CRIPPLED AMERICA low-info years wh_ 
 "@MrTohNey: @dphilbs leaving record in New York. #ImWithYou @realdonaldtrump #Trump! years! Great Poll has already 40%" 
 birthday? just endorsed a taste of @CNN ! He has already saying she is effort with @realDonaldTrump" #ICYMI- miss back so. He was relax! 
 Scott, @realDonaldTrump - Alex pummled. Bad Romney,who call Chafey or wonderful ratings calls via @USSIOWA - coming to #DrainTheSwamp 
 Cruz called how to everyone said little badly. Pathetic! 
 Watching the third NBC high. WHETHER GA 13 VA Fact signed week no chance! 
 morning- you More! 
 The press has the crowd of you! 
 .@DennisDMZ #Trump2016 
 Wow #Yourworld A big Caesars is d

In [8]:
from bleu import *
from rouge_score import *
for i in (2,4,6,8,10):
    print(str(i) +"initial words -- BLEU scores:" )
    bleu_score(i,10,generate_text,model)
    print(str(i) +"initial words -- rouge scores:" )
    rouge_scores(i,10,generate_text,model)
for i in (9,10,18,175):
    print("diferent inital words -- BLEU scores:")
    bleu_score(2,i,generate_text,model)
    print("diferent inital words -- rouge scores:")
    rouge_scores(2,i,generate_text,model)

2initial words -- BLEU scores:
RT @DonaldJTrumpJr:Flags to loves believe your @MikeandDawnNY Sad 
 Watching your @washingtonpost increase signed on @FallonTonight to God too times. Im very a baby into
0.1361428786730173
2initial words -- rouge scores:


AttributeError: 'list' object has no attribute 'split'

In [ ]:
from bleu import *
bleu_score(2,10,generate_text,model)